from "(簡)Mastering Machine Learning With scikit-learn"

### 特徵提取與處理

另外可參考這篇: [如何对文本提取特征](http://m.1supin.com/it/wd/215/194407.html)

做文本分類這樣的問題，需要從大量語料中提取特徵，並將這些文本特徵變換為數值特徵。
對於文本、字符串數據，我們有4種常用方法（Bags of words，TF，TF-IDF，FeatureHasher），將原始數據變為數值型數據。

In [1]:
# 一般分類變量特徵提取
# 分类变量通常用独热编码（One-of-K or One-Hot Encoding），通过二进制数来表示每个解释变量的特征。
from sklearn.feature_extraction import DictVectorizer
onehot_encoder = DictVectorizer()
instances = [{'city': 'New York'},{'city': 'San Francisco'}, {'city': 'Chapel Hill'}]
print onehot_encoder.fit_transform(instances).toarray()

[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]]


### 文字特徵提取
很多機器學習問題涉及自然語言處理（NLP），必然要處理文字信息。文字必須轉換成可以量化的特徵向量。下面介紹最常用的文字表示方法：詞庫模型（Bag-of-words model）。

這種所謂Bags of words的特徵，是將訓練集中所有出現過的單詞做成一個字典，統計每個單詞出現的次數，作為特徵。

詞庫模型是文字模型化的最常用方法。對於一個文檔（document），忽略其詞序和語法，句法，將其僅僅看做是一個詞集合，或者說是詞的一個組合，文檔中每個詞的出現都是獨立的，不依賴於其他詞是否出現，或者說當這篇文章的作者在任意一個位置選擇一個詞彙都不受前面句子的影響而獨立選擇的。詞庫模型可以看成是獨熱編碼的一種擴展，它為每個單詞設值一個特徵值。詞庫模型依據是用類似單詞的文章意思也差不多。詞庫模型可以通過有限的編碼信息實現有效的文檔分類和檢索。
一批文檔的集合稱為文集（corpus）。

In [4]:
# 让我们用一个由两个文档组成的文集来演示词库模型：
corpus = ['UNC played Duke in basketball', 'Duke lost the basketball game']

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
print vectorizer.fit_transform(corpus)
print vectorizer.fit_transform(corpus).todense() 
# 使用todense(), 轉換成數值特徵Bags of words
print vectorizer.vocabulary_

  (0, 0)	1
  (0, 3)	1
  (0, 1)	1
  (0, 5)	1
  (0, 7)	1
  (1, 2)	1
  (1, 6)	1
  (1, 4)	1
  (1, 0)	1
  (1, 1)	1
[[1 1 0 1 0 1 0 1]
 [1 1 1 0 1 0 1 0]]
{u'duke': 1, u'basketball': 0, u'lost': 4, u'played': 5, u'game': 2, u'unc': 7, u'in': 3, u'the': 6}


In [6]:
# 加一個文檔
corpus = ['UNC played Duke in basketball','Duke lost the basketball game','I ate a sandwich'] 
vectorizer = CountVectorizer()
print vectorizer.fit_transform(corpus).todense() 
print vectorizer.vocabulary_

#通过CountVectorizer类可以得出上面的结果。词汇表里面有10个单词，但a不在词汇表里面，是因为a的长度不符合CountVectorizer类的要求。
#CountVectorizer类通过正则表达式用空格分割句子，然后抽取长度大于等于2的字母序列。

[[0 1 1 0 1 0 1 0 0 1]
 [0 1 1 1 0 1 0 0 1 0]
 [1 0 0 0 0 0 0 1 0 0]]
{u'duke': 2, u'basketball': 1, u'lost': 5, u'played': 6, u'in': 4, u'game': 3, u'sandwich': 7, u'unc': 9, u'ate': 0, u'the': 8}


In [9]:
# 計算這三個文檔的歐氏距離 
from sklearn.metrics.pairwise import euclidean_distances
counts = vectorizer.fit_transform(corpus).todense()
for x,y in [[0,1], [0,2], [1,2]]:
    dist = euclidean_distances(counts[x], counts[y])
    print '文檔{}與文檔{}的距離{}'.format(x,y,dist)

# euclidean_distances函数可以计算若干向量的距离，表示两个语义最相似的文档其向量在空间中也是最接近的。

文檔0與文檔1的距離[[ 2.44948974]]
文檔0與文檔2的距離[[ 2.64575131]]
文檔1與文檔2的距離[[ 2.64575131]]


In [11]:
# 停用詞過濾
vectorizer = CountVectorizer(stop_words='english')
print vectorizer.fit_transform(corpus).todense() 
print vectorizer.vocabulary_

[[0 1 1 0 0 1 0 1]
 [0 1 1 1 1 0 0 0]
 [1 0 0 0 0 0 1 0]]
{u'duke': 2, u'basketball': 1, u'lost': 4, u'played': 5, u'game': 3, u'sandwich': 6, u'unc': 7, u'ate': 0}


#### 詞根還原與詞形還原
停用詞去掉之後，可能還會剩下許多詞，還有一種常用的方法就是詞根還原（stemming ）與詞形還原（lemmatization）。

特徵向量裡面的單詞很多都是一個詞的不同形式，比如jumping和jumps都是jump的不同形式。詞根還原與詞形還原就是為了將單詞從不同的時態、派生形式還原。

In [13]:
corpus = [
'He ate the sandwiches',
'Every sandwich was eaten by him'
] 
vectorizer = CountVectorizer(binary=True, stop_words='english')
print(vectorizer.fit_transform(corpus).todense())
print(vectorizer.vocabulary_)

[[1 0 0 1]
 [0 1 1 0]]
{u'sandwich': 2, u'ate': 0, u'sandwiches': 3, u'eaten': 1}


In [14]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [16]:
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
wordnet_tags = ['n', 'v']
corpus = ['He ate the sandwiches','Every sandwich was eaten by him']
stemmer = PorterStemmer()
print('Stemmed:', [[stemmer.stem(token) for token in word_tokenize(document)] for document in corpus])

('Stemmed:', [['He', 'ate', 'the', u'sandwich'], [u'everi', 'sandwich', u'wa', 'eaten', 'by', 'him']])


In [17]:
def lemmatize(token, tag):
    if tag[0].lower() in ['n', 'v']:
        return lemmatizer.lemmatize(token, tag[0].lower())
    return token

lemmatizer = WordNetLemmatizer()
tagged_corpus = [pos_tag(word_tokenize(document)) for document in corpus]
print('Lemmatized:', [[lemmatize(token, tag) for token, tag in document] for document in tagged_corpus])

('Lemmatized:', [['He', u'eat', 'the', u'sandwich'], ['Every', 'sandwich', u'be', u'eat', 'by', 'him']])


### 帶TF-IDF權重的擴展詞庫
前面我們用詞庫模型構建了判斷單詞是個在文檔中出現的特徵向量。這些特徵向量與單詞的語法，順序，頻率無關。

不過直覺告訴我們文檔中單詞的頻率對文檔的意思有重要作用。一個文檔中某個詞多次出現，相比只出現過一次的單詞更能體現反映文檔的意思。現在我們就將單詞頻率加入特徵向量，然後介紹由詞頻引出的兩個問題。

我們用一個整數來代碼單詞的頻率。

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['The dog ate a sandwich, the wizard transfigured a sandwich, and I ate a sandwich']
vectorizer = CountVectorizer(stop_words='english')
print(vectorizer.fit_transform(corpus).todense())
print(vectorizer.vocabulary_)

[[2 1 3 1 1]]
{u'sandwich': 2, u'wizard': 4, u'dog': 1, u'transfigured': 3, u'ate': 0}


結果中第一行是單詞的頻率，dog頻率為1，sandwich頻率為3。

注意和前面不同的是，binary=True沒有了，因為binary默認是False，這樣返回的是詞彙表的詞頻，不是二進制結果[1 1 1 1 1]。

這種單詞頻率構成的特徵向量為文檔的意思提供了更多的信息，但是在對比不同的文檔時，需要考慮文檔的長度。

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
'The dog ate a sandwich and I ate a sandwich',
'The wizard transfigured a sandwich'] 
vectorizer = TfidfVectorizer(stop_words='english')
print(vectorizer.fit_transform(corpus).todense())
print(vectorizer.vocabulary_)

[[ 0.75458397  0.37729199  0.53689271  0.          0.        ]
 [ 0.          0.          0.44943642  0.6316672   0.6316672 ]]
{u'sandwich': 2, u'wizard': 4, u'dog': 1, u'transfigured': 3, u'ate': 0}


#### 通過哈希技巧實現特徵向量

前面我們是用包含文集所有詞塊的詞典來完成文檔詞塊與特徵向量的映射的。這麼做有兩個缺點。

首先是文集需要被調用兩次。第一次是創建詞典，第二次是創建文檔的特徵向量。

另外，詞典必須儲存在內存里，如果文集特別大就會很耗內存。通過哈希表可以有效的解決這些問題。

可以將詞塊用哈希函數來確定它在特徵向量的索引位置，可以不創建詞典，這稱為哈希技巧（hashing trick）。scikitlearn
提供了HashingVectorizer來實現這個技巧：

In [3]:
from sklearn.feature_extraction.text import HashingVectorizer
corpus = ['the', 'ate', 'bacon', 'cat']
vectorizer = HashingVectorizer(n_features=6)
print(vectorizer.transform(corpus).todense())

[[-1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0.  0.]]


哈希技巧是無固定狀態的（stateless），它把任意的數據塊映射到固定數目的位置，並且保證相同的輸入一定產生相同的輸出，不同的輸入盡可能產生不同的輸出。

它可以用並行，線上，流式傳輸創建特徵向量，因為它初始化是不需要文集輸入。n_features是一個可選參數，默認值是2^20，這裡設置成6是為了demo。